## Import Library and dataset

In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

In [24]:
df = pd.read_csv("../data/interests_without_onehot.csv")

features = df.iloc[:, :-1].values
labels = df["y"].values

dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [25]:
for features, label in dataset.take(5):
    print("features :", features)
    print("Label:", label)
    print("Label dtype:", label.dtype)

features : tf.Tensor([0 1 0 2 1 0 0 2 0 1 0 2 0 0 1 1 0 0 2 1 0 1 2 1 1], shape=(25,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([1 0 1 0 0 0 0 0 0 0 0 0 0 2 2 1 1 1 0 2 0 1 2 1 0], shape=(25,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([2 1 2 0 0 1 2 2 0 1 2 0 0 1 2 1 0 0 2 1 0 2 0 1 1], shape=(25,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([0 2 0 0 0 2 2 0 0 0 1 1 0 0 1 1 0 0 2 1 0 2 2 0 1], shape=(25,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([1 1 1 1 1 1 1 1 0 1 2 1 0 1 1 1 1 1 0 1 2 1 1 1 1], shape=(25,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>


## Preprocessing Dataset

In [26]:
keys = tf.constant(["IT sector", "Goverment sector", "Health sector", "Education sector", "Sports sector", "Finance sector", "Entertainment sector"])
values = tf.constant([0, 1, 2, 3, 4, 5, 6])

table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(keys, values),
    default_value=-1
)

def preprocess(features, label):
    label_int = table.lookup(label)
    label_encoded = tf.one_hot(label_int, depth=7)
    
    return features, label_encoded

dataset = dataset.map(preprocess)

In [27]:
for features, labels in dataset.take(5):
    print("Original Features:", features)
    print("One-Hot Encoded Labels:", labels.numpy())

Original Features: tf.Tensor([0 1 0 2 1 0 0 2 0 1 0 2 0 0 1 1 0 0 2 1 0 1 2 1 1], shape=(25,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([1 0 1 0 0 0 0 0 0 0 0 0 0 2 2 1 1 1 0 2 0 1 2 1 0], shape=(25,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([2 1 2 0 0 1 2 2 0 1 2 0 0 1 2 1 0 0 2 1 0 2 0 1 1], shape=(25,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([0 2 0 0 0 2 2 0 0 0 1 1 0 0 1 1 0 0 2 1 0 2 2 0 1], shape=(25,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([1 1 1 1 1 1 1 1 0 1 2 1 0 1 1 1 1 1 0 1 2 1 1 1 1], shape=(25,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]


## Splitting Dataset

In [72]:
features = []
labels = []

for feature, label in dataset:
    features.append(feature.numpy())
    labels.append(label.numpy())

features = np.array(features)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.25, random_state=42, stratify=labels
)


train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

In [73]:
for features, labels in train_dataset.take(1):
    print("Features shape:", features.shape)
    print("Labels shape:", labels.shape)

Features shape: (32, 25)
Labels shape: (32, 7)


## Creating Model

In [91]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(25,), kernel_regularizer=l2(0.01)),
    Dropout(0.4),
    # Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    # Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(7, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

## Training Model

In [92]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=300,
    callbacks=[early_stopping]
)

Epoch 1/300
5/5 [==============================] - 1s 49ms/step - loss: 1.0668 - accuracy: 0.1879 - val_loss: 1.0042 - val_accuracy: 0.1400
Epoch 2/300
5/5 [==============================] - 0s 5ms/step - loss: 0.9926 - accuracy: 0.1611 - val_loss: 0.9378 - val_accuracy: 0.2000
Epoch 3/300
5/5 [==============================] - 0s 6ms/step - loss: 0.9190 - accuracy: 0.2416 - val_loss: 0.8768 - val_accuracy: 0.2800
Epoch 4/300
5/5 [==============================] - 0s 7ms/step - loss: 0.8599 - accuracy: 0.2550 - val_loss: 0.8197 - val_accuracy: 0.3400
Epoch 5/300
5/5 [==============================] - 0s 6ms/step - loss: 0.8018 - accuracy: 0.3624 - val_loss: 0.7669 - val_accuracy: 0.4400
Epoch 6/300
5/5 [==============================] - 0s 6ms/step - loss: 0.7758 - accuracy: 0.2282 - val_loss: 0.7212 - val_accuracy: 0.5000
Epoch 7/300
5/5 [==============================] - 0s 6ms/step - loss: 0.7232 - accuracy: 0.2752 - val_loss: 0.6838 - val_accuracy: 0.5000
Epoch 8/300
5/5 [=========

In [94]:
hasil = model.predict([[1,0,0,0,1,0,1,2,1,0,0,1,1,1,0,0,0,1,1,0,2,1,0,2,2]])
print(f"HASIL : {hasil}")

1/1 [==============================] - 0s 50ms/step
HASIL : [[5.7550706e-04 9.1644210e-01 1.8752669e-03 1.0191994e-03 9.7592667e-02
  1.5732896e-03 7.1725990e-03]]


## Save Model

In [95]:
model.save(filepath="../src/model/sector_classification_model.h5")